In [1]:
%load_ext lab_black

In [22]:
import pandas as pd

from typing import Iterable

from mppsteel.config.model_scenarios import DEFAULT_SCENARIO
from mppsteel.config.model_config import (
    USD_TO_EUR_CONVERSION_DEFAULT,
    PROJECT_PATH,
    PKL_DATA_FORMATTED,
    PKL_DATA_IMPORTS,
    MODEL_YEAR_RANGE,
    MAIN_REGIONAL_SCHEMA,
)

from mppsteel.utility.file_handling_utility import (
    get_scenario_pkl_path,
    read_pickle_folder,
)

from mppsteel.data_preprocessing.investment_cycles import PlantInvestmentCycle
from mppsteel.utility.location_utility import create_country_mapper
from mppsteel.data_load_and_format.country_reference import country_df_formatter
from mppsteel.model_solver.tco_and_abatement_optimizer import subset_presolver_df
from mppsteel.model_solver.solver_constraints import read_and_format_tech_availability
from mppsteel.data_load_and_format.steel_plant_formatter import create_active_check_col

from mppsteel.model_solver.solver import choose_technology

In [3]:
scenario_dict = DEFAULT_SCENARIO.copy()
scenario_dict["usd_to_eur"] = USD_TO_EUR_CONVERSION_DEFAULT
scenario_dict["eur_to_usd"] = 1.0 / scenario_dict["usd_to_eur"]

In [4]:
class ChooseTechnologyInput:
    def __init__(
        self,
        *,
        original_plant_df: pd.DataFrame = pd.DataFrame(),
        year_range: Iterable[int] = [],
        tech_moratorium: bool = False,
        trade_active: bool = False,
        enforce_constraints: bool = False,
        regional_scrap_constraint: bool = False,
        plant_investment_cycle_container: PlantInvestmentCycle = PlantInvestmentCycle(),
        variable_costs_regional: pd.DataFrame = pd.DataFrame(),
        country_ref: pd.DataFrame = pd.DataFrame(),
        rmi_mapper: dict[str, str] = {},
        country_ref_f: pd.DataFrame = pd.DataFrame(),
        bio_constraint_model: pd.DataFrame = pd.DataFrame(),
        co2_constraint: pd.DataFrame = pd.DataFrame(),
        ccs_constraint: pd.DataFrame = pd.DataFrame(),
        steel_demand_df: pd.DataFrame = pd.DataFrame(),
        tech_availability: pd.DataFrame = pd.DataFrame(),
        ta_dict: dict[str, int] = {},
        capex_dict: dict[str, pd.DataFrame] = {},
        business_case_ref: dict[tuple[str, str], float] = {},
        green_premium_timeseries: pd.DataFrame = pd.DataFrame(),
        tco_summary_data: pd.DataFrame = pd.DataFrame(),
        tco_slim: pd.DataFrame = pd.DataFrame(),
        levelized_cost: pd.DataFrame = pd.DataFrame(),
        steel_plant_abatement_switches: pd.DataFrame = pd.DataFrame(),
        abatement_slim: pd.DataFrame = pd.DataFrame(),
        scenario_dict: dict[str, any] = {},
    ):
        self.original_plant_df = original_plant_df
        self.year_range = year_range
        self.tech_moratorium = tech_moratorium
        self.trade_active = trade_active
        self.enforce_constraints = enforce_constraints
        self.regional_scrap_constraint = regional_scrap_constraint
        self.plant_investment_cycle_container = plant_investment_cycle_container
        self.variable_costs_regional = variable_costs_regional
        self.country_ref = country_ref
        self.rmi_mapper = rmi_mapper
        self.country_ref_f = country_ref_f
        self.bio_constraint_model = bio_constraint_model
        self.co2_constraint = co2_constraint
        self.ccs_constraint = ccs_constraint
        self.steel_demand_df = steel_demand_df
        self.tech_availability = tech_availability
        self.ta_dict = ta_dict
        self.capex_dict = capex_dict
        self.business_case_ref = business_case_ref
        self.green_premium_timeseries = green_premium_timeseries
        self.tco_summary_data = tco_summary_data
        self.tco_slim = tco_slim
        self.levelized_cost = levelized_cost
        self.steel_plant_abatement_switches = steel_plant_abatement_switches
        self.abatement_slim = abatement_slim
        self.scenario_dict = scenario_dict

    @classmethod
    def from_filesystem(
        cls, scenario_dict, project_dir=PROJECT_PATH, year_range=MODEL_YEAR_RANGE
    ):
        tech_moratorium = scenario_dict["tech_moratorium"]
        trade_active = scenario_dict["trade_active"]
        enforce_constraints = scenario_dict["enforce_constraints"]
        regional_scrap_constraint = scenario_dict["regional_scrap_constraint"]

        intermediate_path = get_scenario_pkl_path(
            scenario_dict["scenario_name"], "intermediate"
        )
        original_plant_df = read_pickle_folder(
            PROJECT_PATH / PKL_DATA_FORMATTED, "steel_plants_processed", "df"
        )
        plant_investment_cycle_container = read_pickle_folder(
            PROJECT_PATH / PKL_DATA_FORMATTED, "plant_investment_cycle_container", "df"
        )
        variable_costs_regional = read_pickle_folder(
            PROJECT_PATH / intermediate_path, "variable_costs_regional", "df"
        )
        country_ref = read_pickle_folder(
            PROJECT_PATH / PKL_DATA_IMPORTS, "country_ref", "df"
        )
        rmi_mapper = create_country_mapper(path=str(PROJECT_PATH / PKL_DATA_IMPORTS))
        country_ref_f = country_df_formatter(country_ref)

        bio_constraint_model = read_pickle_folder(
            PROJECT_PATH / intermediate_path, "bio_constraint_model_formatted", "df"
        )
        co2_constraint = read_pickle_folder(
            PROJECT_PATH / PKL_DATA_IMPORTS, "ccs_co2", "df"
        )
        ccs_constraint = read_pickle_folder(
            PROJECT_PATH / intermediate_path, "ccs_constraints_model_formatted", "df"
        )
        steel_demand_df = read_pickle_folder(
            PROJECT_PATH / intermediate_path, "regional_steel_demand_formatted", "df"
        )
        tech_availability_raw = read_pickle_folder(
            PROJECT_PATH / PKL_DATA_IMPORTS, "tech_availability", "df"
        )
        ta_dict = dict(
            zip(
                tech_availability_raw["Technology"],
                tech_availability_raw["Year available from"],
            )
        )
        tech_availability = read_and_format_tech_availability(tech_availability_raw)
        capex_dict = read_pickle_folder(
            PROJECT_PATH / PKL_DATA_FORMATTED, "capex_dict", "dict"
        )
        business_case_ref = read_pickle_folder(
            PROJECT_PATH / PKL_DATA_FORMATTED, "business_case_reference", "df"
        )
        green_premium_timeseries = read_pickle_folder(
            PROJECT_PATH / intermediate_path, "green_premium_timeseries", "df"
        ).set_index("year")
        tco_summary_data = read_pickle_folder(
            PROJECT_PATH / intermediate_path, "tco_summary_data", "df"
        )
        tco_slim = subset_presolver_df(tco_summary_data, subset_type="tco_summary")
        levelized_cost = read_pickle_folder(
            PROJECT_PATH / intermediate_path, "levelized_cost", "df"
        )
        levelized_cost["region"] = levelized_cost["country_code"].apply(
            lambda x: rmi_mapper[x]
        )
        steel_plant_abatement_switches = read_pickle_folder(
            PROJECT_PATH / intermediate_path, "emissivity_abatement_switches", "df"
        )
        abatement_slim = subset_presolver_df(
            steel_plant_abatement_switches, subset_type="abatement"
        )
        return cls(
            original_plant_df=original_plant_df,
            year_range=year_range,
            tech_moratorium=tech_moratorium,
            trade_active=trade_active,
            enforce_constraints=enforce_constraints,
            regional_scrap_constraint=regional_scrap_constraint,
            plant_investment_cycle_container=plant_investment_cycle_container,
            variable_costs_regional=variable_costs_regional,
            country_ref=country_ref,
            rmi_mapper=rmi_mapper,
            country_ref_f=country_ref_f,
            bio_constraint_model=bio_constraint_model,
            co2_constraint=co2_constraint,
            ccs_constraint=ccs_constraint,
            steel_demand_df=steel_demand_df,
            tech_availability=tech_availability,
            ta_dict=ta_dict,
            capex_dict=capex_dict,
            business_case_ref=business_case_ref,
            green_premium_timeseries=green_premium_timeseries,
            tco_summary_data=tco_summary_data,
            tco_slim=tco_slim,
            levelized_cost=levelized_cost,
            steel_plant_abatement_switches=steel_plant_abatement_switches,
            abatement_slim=abatement_slim,
            scenario_dict=scenario_dict,
        )

In [5]:
cti = ChooseTechnologyInput.from_filesystem(scenario_dict)
print(cti)

2022-06-07 12:57:33,779 — mppsteel.data_load_and_format.country_reference — INFO — Formatting country DataFrame


In [6]:
cti.scenario_dict

{'scenario_name': 'default',
 'tech_moratorium': True,
 'enforce_constraints': True,
 'transitional_switch': True,
 'carbon_tax_scenario': 'off',
 'green_premium_scenario': 'off',
 'electricity_cost_scenario': 'average',
 'grid_scenario': 'low',
 'hydrogen_cost_scenario': 'low',
 'biomass_cost_scenario': 'average',
 'ccs_cost_scenario': 'low',
 'ccs_capacity_scenario': 'low',
 'fossil_fuel_scenario': 'high',
 'steel_demand_scenario': 'average',
 'tech_switch_scenario': 'lowest_cost',
 'solver_logic': 'scale',
 'trade_active': True,
 'regional_scrap_constraint': True,
 'usd_to_eur': 0.877,
 'eur_to_usd': 1.1402508551881414}

In [18]:
MAIN_REGIONAL_SCHEMA

'rmi_region'

In [20]:
cti.original_plant_df[MAIN_REGIONAL_SCHEMA].unique()

array(['Africa', 'RoW', 'Southeast Asia', 'China', 'India',
       'Japan, South Korea, and Taiwan', 'Middle East',
       'South and Central America', 'CIS', 'Europe', 'NAFTA'],
      dtype=object)

In [27]:
year = 2020
year_start_df = pd.DataFrame(
    [["STE02421", "Africa", True, 0.0, "status"]],
    columns=["plant_id", "rmi_region", "active_check", "plant_capacity", "status"],
)
year_start_df.apply(create_active_check_col, year=year, axis=1)

0    False
dtype: bool

In [24]:
year = 2020
year_start_df = cti.original_plant_df.copy()
year_start_df.apply(create_active_check_col, year=year, axis=1)

0       True
1       True
2       True
4       True
7       True
        ... 
1375    True
1376    True
1377    True
1382    True
1383    True
Length: 974, dtype: bool

In [143]:
def choose_technology_new(cti: ChooseTechnologyInput) -> dict:
    tech_moratorium = cti.tech_moratorium
    trade_scenario = cti.trade_active
    enforce_constraints = cti.enforce_constraints
    regional_scrap = cti.regional_scrap_constraint
    original_plant_df = cti.original_plant_df
    PlantInvestmentCycleContainer = cti.plant_investment_cycle_container
    variable_costs_regional = cti.variable_costs_regional
    country_ref = cti.country_ref
    rmi_mapper = cti.rmi_mapper
    country_ref_f = cti.country_ref_f
    bio_constraint_model = cti.bio_constraint_model
    co2_constraint = cti.co2_constraint
    ccs_constraint = cti.ccs_constraint
    steel_demand_df = cti.steel_demand_df
    tech_availability = cti.tech_availability
    ta_dict = cti.ta_dict
    capex_dict = cti.capex_dict
    business_case_ref = cti.business_case_ref
    green_premium_timeseries = cti.green_premium_timeseries
    tco_summary_data = cti.tco_summary_data
    tco_slim = cti.tco_slim
    levelized_cost = cti.levelized_cost
    steel_plant_abatement_switches = cti.steel_plant_abatement_switches
    abatement_slim = cti.abatement_slim

    # Initialize plant container
    PlantIDC = PlantIdContainer()
    PlantIDC.add_steel_plant_ids(original_plant_df)
    year_start_df = original_plant_df.copy()

In [217]:
foo = 1, 2, 3, {}

In [218]:
foo[-1]["blub"] = "bla"

In [219]:
foo

(1, 2, 3, {'blub': 'bla'})

In [223]:
MAIN_REGIONAL_SCHEMA

'rmi_region'